In [1]:
from smb.SMBConnection import SMBConnection
import socket
import pathlib
import yaml
import os
import shutil

In [2]:
# Path to credentials
desktop_path = pathlib.Path.home()/'Desktop'
path_to_config_file = desktop_path.joinpath('config.yml')

In [3]:
# Loading credentials
conf = yaml.safe_load(open(str(path_to_config_file)))
usr = conf['user']['username']
pwd = conf['user']['password']
remote_address = conf['user']['remote_address']
domain = conf['user']['domain']

In [4]:
# LOCAL NAME
local_name = socket.gethostbyname(socket.gethostname())
# SERVER NAME
share_name = 'share'
server_name, alias, addresslist = socket.gethostbyaddr(remote_address)

In [12]:
folder_path = '/Test/test_dir/'
# Creating a local directory in the local computer
temp_local_path = pathlib.Path().absolute().joinpath('temp')
if os.path.exists(str(temp_local_path)):
    shutil.rmtree(temp_local_path)
os.makedirs(str(temp_local_path))
# Establishing connection to the server
conn = SMBConnection(username=usr, password=pwd, domain=domain, my_name=local_name, use_ntlm_v2 = False, remote_name=server_name, is_direct_tcp=False)
#print(server_name)
#is_connected = conn.connect(server_name,timeout=60)
is_connected = conn.connect(server_name)

ProtocolError: Unknown status value (0xC0000022) in SMB_COM_SESSION_SETUP_ANDX (with extended security)
==================== SMB Message ====================
Command: 0x01 (SMB2_COM_SESSION_SETUP) 
Status: 0xC0000022 
Flags: 0x01 
PID: 15218 
MID: 2 
TID: 0 
Data: 9 bytes 
b'090000000000000000' 
==================== SMB Data Packet (hex) ====================
b'fe534d4240000000220000c00100010001000000000000000200000000000000723b0000000000006784e5fe0000000000000000000000000000000000000000090000000000000000'


In [ ]:
raise

In [ ]:
folder_path = '/Test/test_dir/'
# Creating a local directory in the local computer
temp_local_path = pathlib.Path().absolute().joinpath('temp')
if os.path.exists(str(temp_local_path)):
    shutil.rmtree(temp_local_path)
os.makedirs(str(temp_local_path))
# Establishing connection to the server
conn = SMBConnection(username=usr, password=pwd, domain=domain, my_name=local_name, remote_name=server_name, is_direct_tcp=True,use_ntlm_v2=True)
print(server_name)
#is_connected = conn.connect(server_name,timeout=60)
is_connected = conn.connect(server_name)

print('Connection established: ', is_connected)
print('The files are downloaded to:', temp_local_path)

# Iterate in the folder to download all tif files
list_dir = conn.listPath(share_name, folder_path)
for file in list_dir:
    if (file.filename not in ['.', '..']) and ('.tif' in file.filename) :
        print ('File Downloaded :', file.filename)
        fileobj = open(file.filename,'wb')
        conn.retrieveFile(share_name,folder_path+file.filename,fileobj)
        # moving files in the local computer
        shutil.move(pathlib.Path().absolute().joinpath(file.filename), temp_local_path.joinpath(file.filename))
conn.close()

In [ ]:

list_names = [file.filename for file in list_dir]
list_names

In [ ]:
class NASConnection():
    '''
    This class is intended to establish a connection between a Network-Attached storage and a local computer. The class allow the user to establish a connection to NAS, download specific files, and write back files to NAS.
    This class doesn't allow the user to delete, modify or overwrite files in NAS.

    To use this class you need to:
    1) Use the university's network or use the two factor authentication to connect to the university's VPN.
    2) You need to create a configuration yaml file, with the following format:

    user:
        username: name_of_the_user_in_the_nas_server
        password: user_password_in_the_nas_server 
        remote_address : ip or name for the nas server
        domain: domain for the nas server
    
    Parameters
    --  --  --  --  -- 

    ,: bool, optional
        parameter description. The default is True. 

    path_to_config_file : str, Pathlib obj
        The path in the local computer that containts the config file.
    share_name : str
        Name of the share partition to access in NAS. The default is 'share'.
    '''
    def __init__(self,path_to_config_file,share_name = 'share'):
        # Loading credentials
        conf = yaml.safe_load(open(str(path_to_config_file)))
        usr = conf['user']['username']
        pwd = conf['user']['password']
        remote_address = conf['user']['remote_address']
        domain = conf['user']['domain']
        # LOCAL NAME
        local_name = socket.gethostbyname(socket.gethostname())
        # SERVER NAME
        self.share_name = share_name
        self.server_name, alias, addresslist = socket.gethostbyaddr(remote_address)
        # Deffining the connection to NAS
        self.conn = SMBConnection(username=usr, password=pwd, domain=domain, my_name=local_name, remote_name=self.server_name, is_direct_tcp=True)
    def connect_to_server(self,timeout=60):
        is_connected = self.conn.connect(self.server_name,timeout=timeout)
        if is_connected == True:
            print('Connection established')
        else:
            print('Connection failed')
        return self.conn
        
    def copy_files(self, remote_folder_path, local_folder_path, timeout=60):
        '''
        This method downloads the tif files from NAS to a temp folder in the local computer

        Parameters
        --  --  --  --  -- 

        remote_folder_path : str, Pathlib obj
            The path in the remote folder to download
        local_folder_path : str, Pathlib obj
            The path in the local computer where the files will be copied
        timeout : int, optional
            seconds to establish connection. The default is 60.
        '''
        # Connecting to NAS
        is_connected = self.conn.connect(self.server_name,timeout=timeout)
        if is_connected == True:
            print('Connection established')
        else:
            print('Connection failed')
        # Converting the paths to a Pathlib object
        if type(local_folder_path) == str:
            local_folder_path = pathlib.Path(local_folder_path)
        if type(remote_folder_path)==str:
            remote_folder_path = pathlib.Path(remote_folder_path)
        # Creating a temporal folder in path
        if  not str(local_folder_path)[-4:] ==  'temp':
            local_folder_path = pathlib.Path(local_folder_path).joinpath('temp')
        # Making the local directory
        if os.path.exists(str(local_folder_path)):
            shutil.rmtree(local_folder_path)
        os.makedirs(str(local_folder_path))
        
        # Iterate in the folder to download all tif files
        list_dir = self.conn.listPath(self.share_name, str(remote_folder_path))
        for file in list_dir:
            if (file.filename not in ['.', '..']) and ('.tif' in file.filename) :
                print ('File Downloaded :', file.filename)
                fileobj = open(file.filename,'wb')
                self.conn.retrieveFile(self.share_name, str( pathlib.Path(remote_folder_path).joinpath(file.filename) ),fileobj)
                # moving files in the local computer
                
                #shutil.move(pathlib.Path().absolute().joinpath(file.filename), local_folder_path.joinpath(file.filename))
    
    def write_files_to_NAS(self, file_to_write_to_NAS, remote_folder_path,  timeout=60):
        '''
        This method writes files from a local computer to NAS 

        Parameters
        --  --  --  --  -- 

        remote_folder_path : str, Pathlib obj
            The path in the remote folder to download
        local_folder_path : str, Pathlib obj
            The path in the local computer where the files will be copied
        timeout : int, optional
            seconds to establish connection. The default is 60.
        '''
        # Connecting to NAS
        is_connected = self.conn.connect(self.server_name,timeout=timeout)
        if is_connected == True:
            print('Connection established')
        else:
            print('Connection failed')
        # Converting the paths to a Pathlib object
        if type(file_to_write_to_NAS) == str:
            file_to_write_to_NAS = pathlib.Path(file_to_write_to_NAS)

        if type(remote_folder_path)==str:
            remote_folder_path = pathlib.Path(remote_folder_path)

        # checks that the file doesn't exist on NAS. If it exist it will create a new name as follows original_name__1
        # Iterate in the folder to download all tif files
        list_dir = self.conn.listPath(self.share_name, str(remote_folder_path))
        list_all_files_in_NAS = [file.filename for file in list_dir]

        for file in list_dir:
            if str(file_to_write_to_NAS.name) not in list_all_files_in_NAS:

                #self.conn.storeFile(self.share_name, ,path,file)

    

In [ ]:
conn_session= NASConnection(path_to_config_file).connect_to_server()

In [ ]:
remote_folder_path = '/Test/test_dir/'
local_folder_path = pathlib.Path().absolute().joinpath('temp')
NASConnection(path_to_config_file).copy_files(remote_folder_path, local_folder_path,timeout=60)

In [ ]:
conn_session.close()

In [ ]:
# https://github.com/humberry/smb-example/blob/master/smb-test.py